## **To recreate study results please load package versions found in main_requirements.txt**

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

sc.settings.verbosity = 1
sc.logging.print_header()

scanpy==1.6.0 anndata==0.7.4 umap==0.4.3 numpy==1.18.2 scipy==1.4.1 pandas==1.0.3 scikit-learn==0.22.2.post1 statsmodels==0.10.1 python-igraph==0.8.0 louvain==0.6.1 leidenalg==0.8.0


### Concatenate all files into one anndata object

In [2]:
# directory path where 10x filtered h5's are located
from os import listdir
data_in_path = "/home/cherring/working_data_03/data/cellranger_outputs/brain_maturation_RNA/final_set_filtered_h5s/"
# file formats should be: RUNID_AGE_Chemistry_filetypename.h5
file_list = np.sort( listdir( data_in_path))
file_list[:2]

array(['RL1612_34d_v2_filtered.h5', 'RL1613_2yr_v2_filtered.h5'],
      dtype='<U26')

In [3]:
# concatenate all data into one anndata
adata_list = []
for fl_itr in file_list:
    fl_ids = fl_itr.split('_')
    adata_itr = sc.read_10x_h5( data_in_path+fl_itr, genome=None)
    # drop genome var feature due to poor handling with concatentate, hg19 was used for all runs
    adata_itr.var.drop( labels='genome', axis=1, inplace=True)
    adata_itr.var['non-unique_names'] = adata_itr.var_names
    adata_itr.var_names_make_unique()
    adata_itr.obs['batch'] = ("_").join( fl_ids[:3])
    adata_itr.obs['RL#']  = fl_ids[0]
    adata_itr.obs['age']  = fl_ids[1]
    adata_itr.obs['chem'] = fl_ids[2]
    adata_list.append( adata_itr)
adata = sc.AnnData.concatenate( *adata_list, join='inner', batch_key="concat_id")
# reformat barcodes to include seq and sample id to deal with any barcode clashes
barcodes = np.array( [ii.split('-')[0] + '-' + jj for ii, jj in zip( adata.obs_names, adata.obs['batch'])])
adata.obs_names = barcodes

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Vari

In [4]:
adata.shape

(194703, 32738)

### Format and add features of interest

In [5]:
# add batch and age order for plotting and etc
adata.uns['batch_order'] = ['RL2103_ga22_v3', 'RL2107_ga24_v3', 'RL2121_ga34_v3', 'RL1777_2d_v3',   'RL1612_34d_v2',  'RL2100_86d_v3',
                            'RL2104_118d_v3', 'RL2108_179d_v3', 'RL2122_301d_v3', 'RL2125_422d_v3', 'RL2105_627d_v3', 'RL1613_2yr_v2', 
                            'RL1786_2yr_v3',  'RL2129_3yr_v3',  'RL2109_4yr_v3',  'RL2106_6yr_v3',  'RL1614_8yr_v2',  'RL2110_10yr_v3',
                            'RL2126_10yr_v3', 'RL2127_12yr_v3', 'RL2130_14yr_v3', 'RL2102_16yr_v3', 'RL2131_17yr_v3', 'RL2123_20yr_v3', 
                            'RL2128_20yr_v3', 'RL2132_25yr_v3', 'RL2124_40yr_v3']
# set age order
adata.uns['age_order'] = [bb.split('_')[1] for bb in adata.uns['batch_order']]

Get numerical age and add developmental stage

In [6]:
# Numerical age for fetal samples are negative based on 40 week gestation, with -280/365 being conception and 0 being birth
def calc_fetal( ga):
    return( ( ( ga * 7) - ( 40 * 7)) / 365)

import re
# convert all age to a common numerical scale and add feature, i.e. years
def numerical_age( adata):
    # get numerical age
    str_age = adata.uns['age_order']
    num_age = []
    for age_itr in str_age:
        digit = int(re.findall('\d+', age_itr)[0])
        if( "ga" in age_itr):
            num_age.append( calc_fetal( digit))
        elif( "d" in age_itr):
            num_age.append( digit / 365)
        elif( "yr" in age_itr):
            num_age.append( digit)
        else:
            print( age_itr + " of unknown age type, numerical value set to 0.0")
    # add feature to anndata
    adata.obs['numerical_age'] = [num_age[str_age.index(ii)] for ii in adata.obs['age']]
    return

In [7]:
# stage of maturation, 
# ref https://www.nature.com/articles/nature10523/tables/1
def stage_id( adata):
    numer_age = adata.obs['numerical_age'].values
    # array to hold stage ids
    stage_id  = np.zeros_like( adata.obs['batch'].values.astype(str))
    stage_id[:] = '!INVALID AGE!'
    
    # Fetal - up to ga38
    stage_id[numer_age<calc_fetal(38)] = 'Fetal'
    # Neonatal - ga38 up to 60d
    stage_id[(numer_age>=calc_fetal(38)) & (numer_age<60/365)] = 'Neonatal'
    # Infancy - 60d up to 1yr 
    stage_id[(numer_age>=60/365) & (numer_age<1)] = 'Infancy'
    # Childhood - 1yr up to 6yr
    stage_id[(numer_age>=1) & (numer_age<10)] = 'Childhood'
    # Adolescence - 12yr up 20yr
    stage_id[(numer_age>=10) & (numer_age<20)] = 'Adolescence'
    # Adulthood - 20yr up to 40
    stage_id[(numer_age>=20) & (numer_age<=40)] = 'Adult'
    # Late Adulthood - 40yr and up 
    stage_id[numer_age>40] = 'Late Adult'
    
    adata.obs['stage_id'] = stage_id
    return

In [8]:
numerical_age( adata)
stage_id( adata)

In [9]:
# set column type to categorical
adata.obs['stage_id'] = adata.obs['stage_id'].astype('category')
# set order and colors of stages
adata.uns['stage_order'] =  ['Fetal', 'Neonatal', 'Infancy', 'Childhood', 'Adolescence', 'Adult']
# add colors and color dict to be used for plotting
stage_ordered_colors = ["#512568", "#443682", "#3D6B93", "#20988C", "#98CA43", "#F9E51B"]
adata.uns['stage_colors_dict'] = dict( zip( adata.uns['stage_order'], stage_ordered_colors))
adata.uns['stage_id_colors'] = [adata.uns['stage_colors_dict'][ii] for ii in adata.obs['stage_id'].cat.categories]

Add sample related features

In [10]:
sample_metrics_df = pd.read_csv( '../data/Sample_Metrics.csv', header=[0], index_col=[0])
sample_metrics_df.head()

,RL#,Sample ID,Age,Sex,Race,PMI,Brain Regions*,Cause of Death,ICD-10 Code,ICD-10 category,Oxygen/No Oxygen,Date-of-Collection,Collection_year,Library Prep Date,Library Prep Lot
1,RL2103,1085,ga22,M,AA,5,BA9,Respiratory Insufficiency,108,Perinatal,Oxygen,NaN,NaN,28.03.2020,1
2,RL2107,4320,ga24,M,AA,9,BA9,"Prematurity, Hyaline membrane disease",108,Perinatal,No Oxygen,NaN,NaN,28.03.2020,1
3,RL2121,5900,ga34,F,White,23,BA9,Hypoplatic Left Heart Syndrome,108,Perinatal,No Oxygen,11.01.2015,2015.0,16.04.2020,2
5,RL1777,4375,2d,F,White,26,BA8,Positional asphyxia,R090,Asphyxia,Oxygen,7.22.2013,2013.0,27.06.2019,4
6,RL1612,4353,34d,M,AA,5,BA9,Probably overlay,R090,Asphyxia,Oxygen,10.04.2011,2011.0,20.11.2018,5


In [11]:
# save index to reset after merge
obs_names = adata.obs_names
# fill values in adata.obs with metrics
adata.obs = adata.obs.merge( sample_metrics_df, how='left')
# remove duplicate and unwanted columns
adata.obs.drop( labels=['Sample ID','Age'], axis='columns', inplace=True)
# add back in BCs after being stripped by merge
adata.obs_names = obs_names
adata.obs.tail()

AnnData expects .obs.index to contain strings, but your first indices are: Int64Index([0, 1], dtype='int64'), …


,batch,RL#,age,chem,concat_id,numerical_age,stage_id,Sex,Race,PMI,Brain Regions*,Cause of Death,ICD-10 Code,ICD-10 category,Oxygen/No Oxygen,Date-of-Collection,Collection_year,Library Prep Date,Library Prep Lot
TTTGTTGCACCGCTGA-RL2132_25yr_v3,RL2132_25yr_v3,RL2132,25yr,v3,26,25.0,Adult,F,AA,24,BA10,Occlusive Pulmonary Thromboembolism,126,Circulatory System,No oxygen,04.30.2014,2014.0,24.04.2020,8
TTTGTTGCAGAATGTA-RL2132_25yr_v3,RL2132_25yr_v3,RL2132,25yr,v3,26,25.0,Adult,F,AA,24,BA10,Occlusive Pulmonary Thromboembolism,126,Circulatory System,No oxygen,04.30.2014,2014.0,24.04.2020,8
TTTGTTGGTAAGGTCG-RL2132_25yr_v3,RL2132_25yr_v3,RL2132,25yr,v3,26,25.0,Adult,F,AA,24,BA10,Occlusive Pulmonary Thromboembolism,126,Circulatory System,No oxygen,04.30.2014,2014.0,24.04.2020,8
TTTGTTGGTTCGGCTG-RL2132_25yr_v3,RL2132_25yr_v3,RL2132,25yr,v3,26,25.0,Adult,F,AA,24,BA10,Occlusive Pulmonary Thromboembolism,126,Circulatory System,No oxygen,04.30.2014,2014.0,24.04.2020,8
TTTGTTGTCGTCCTCA-RL2132_25yr_v3,RL2132_25yr_v3,RL2132,25yr,v3,26,25.0,Adult,F,AA,24,BA10,Occlusive Pulmonary Thromboembolism,126,Circulatory System,No oxygen,04.30.2014,2014.0,24.04.2020,8


### Save anndata to h5ad

In [12]:
adata.write( "../data/combined_count_matrices.h5ad")

... storing 'batch' as categorical
... storing 'RL#' as categorical
... storing 'age' as categorical
... storing 'chem' as categorical
... storing 'Sex' as categorical
... storing 'Race' as categorical
... storing 'Brain Regions*' as categorical
... storing 'Cause of Death' as categorical
... storing 'ICD-10 Code' as categorical
... storing 'ICD-10 category' as categorical
... storing 'Oxygen/No Oxygen' as categorical
... storing 'Date-of-Collection' as categorical
... storing 'Library Prep Date' as categorical
... storing 'feature_types' as categorical
... storing 'non-unique_names' as categorical
